# Multiprocessing

Reference: https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming

In [17]:
import multiprocessing
from tqdm import tqdm
from functools import partial
import os
import sys
from pytubefix import YouTube
import psutil
import time
import subprocess

### Parallelism Concept

In [18]:
def say_numbers():
    # Count 1~5
    for i in range(1,6,1):
        print(f"Number: #{i}")
        #sys.stdout.write(f"Number: #{i}\n")
        #sys.stdout.flush()
        time.sleep(0.1)

def say_alphabet():
    # Count a~e
    for i in range(ord("a"), ord("f")):
        print(f"Alphabet: {chr(i)}")
        #sys.stdout.write(f"Alphabet: {chr(i)}\n")
        #sys.stdout.flush()
        time.sleep(0.1)

In [19]:
print("-"*10+"Sequential execution"+"-"*10)
seq = time.time()
say_numbers()
say_alphabet()
print("execution time :", time.time() - seq)

----------Sequential execution----------
Number: #1
Number: #2
Number: #3
Number: #4
Number: #5
Alphabet: a
Alphabet: b
Alphabet: c
Alphabet: d
Alphabet: e
execution time : 1.0016398429870605


In [20]:
print("-"*10+"Parallel execution"+"-"*10)
par = time.time()   # 러닝타임 체크 시작
p1 = multiprocessing.Process(target=say_numbers)
p2 = multiprocessing.Process(target=say_alphabet)
p1.start()
p2.start()

# join으로 대기하지 않으면 부모 process가 종료되어 자식이 zombie가 된다.
p1.join()
p2.join()

print("execution time :", time.time() - par)    # 러닝타임 체크 끝

----------Parallel execution----------
Number: #1
Alphabet: a
Number: #2
Alphabet: b
Number: #3
Alphabet: c
Number: #4
Alphabet: d
Number: #5
Alphabet: e
execution time : 0.5162351131439209


### Process control methods

- **start()**: Sub process를 실행시킵니다.
- **terminate()**: Process에게 SIGTERM을 보냅니다.
- **join()**: Process가 종료될 때까지 기다립니다.
- join([_timeout_]): 지정된 시간(초 단위)까지 기다립니다.

In [21]:
# terminate() example
# 아래의 코드는 좀비 프로세스를 생성시킵니다.
# `watch -n1 "ps aux|grep defunct"` 명령어로 관찰 할 수 있습니다.

def child_process_func():
    print("Child process counting started")
    # Count 1~5
    for i in range(10):
        print(f"Number: #{i}")
        time.sleep(0.5)
    # print on normal termination only.
    print("Normal termination")


p1 = multiprocessing.Process(target=child_process_func)
print("Process child process")
p1.start()
time.sleep(1)

# Force kill process
p1.terminate()

Process child process
Child process counting started
Number: #0
Number: #1


In [22]:
# join()을 이용해서 좀비 프로세스가 발생하는 것을 방지 할 수 있습니다.

def say_numbers():
    # Count 1~5
    for i in range(1,6,1):
        print(f"Number: #{i}")
        time.sleep(0.1)
    print("Process terminating...", flush=True)

p1 = multiprocessing.Process(target=say_numbers)
print("Process starting...")
p1.start()

# Wait until process terminates
print("Waiting for process to be terminated...")
p1.join()

Process starting...
Number: #1
Number: #2
Waiting for process to be terminated...
Number: #3
Number: #4
Number: #5
Process terminating...


### IPC practice: Popen

In [23]:
print("Current process ID:{}".format(os.getpid()), flush=True)

ps_process = subprocess.Popen(['bash', 'subprocess.sh'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = ps_process.communicate()

print(stdout.decode())

Current process ID:20696
Subprocess PID: 21387, PPID: 20696
total 4350428
drwxrwxr-x  3 eleo95 eleo95       4096  9월 24 09:46 Arduino
drwxrwxr-x  7 eleo95 eleo95       4096  9월  9 10:23 Bare_repo
drwxr-xr-x  2 eleo95 eleo95       4096  9월 19 09:31 Desktop
drwxr-xr-x  3 eleo95 eleo95       4096  9월  9 09:38 Documents
drwxr-xr-x  4 eleo95 eleo95       4096 10월  2 09:28 Downloads
drwxr-xr-x  2 eleo95 eleo95       4096  9월  7 08:36 Music
drwxrwxr-x  3 eleo95 eleo95       4096  9월  9 10:22 Non_bare_repo
drwxr-xr-x  3 eleo95 eleo95       4096  9월 12 16:13 Pictures
drwxr-xr-x  2 eleo95 eleo95       4096  9월  7 08:36 Public
-rw-r--r--  1 eleo95 eleo95        134  1월 30  2023 README.dataset.txt
-rw-r--r--  1 eleo95 eleo95       1072  1월 30  2023 README.roboflow.txt
drwxr-xr-x  2 eleo95 eleo95       4096  9월  7 08:36 Templates
drwxr-xr-x  3 eleo95 eleo95       4096  9월 27 14:45 Videos
drwxrwxr-x  3 eleo95 eleo95       4096  9월 24 09:42 arduino
drwxrwxr-x  5 eleo95 eleo95       4096  9월 13 15:21 

### `pytube` `tqdm` example

- UI와 다운로드가 message_queue로 통신

- UI Process

In [28]:
# UI 프로세스
def draw_ui(message_queue):
    print(
        "UI process starting ... PID:{}, PPID:{}"
        .format(
            os.getpid(),
            psutil.Process(os.getpid()).ppid()), flush=True)
    prev = 0
    tqdm_bar = None
    while True:
        message = message_queue.get()
        if message["type"] == "on_progress":
            if tqdm_bar is None:
                tqdm_bar = tqdm(total=100, desc="Downloading...")
            cur_rate = message["progress_rate"]
            tqdm_bar.update(int(cur_rate-prev))
            prev = int(cur_rate)
        elif message["type"] == "on_complete":
            if tqdm_bar is None:
                tqdm_bar = tqdm(total=100, desc="Downloading...")
            tqdm_bar.update(100-prev)
            tqdm_bar.close()
            print("Completed!")
            break

- Download Process

In [29]:
# 다운로드 프로세스
def on_progress(stream, chunk, bytes_remaining, message_queue):
    total_size = stream.filesize
    bytes_downloaded = total_size - bytes_remaining
    progress = (bytes_downloaded / total_size) * 100
    message_queue.put({"type":"on_progress", "progress_rate":progress})

def on_complete(stream, file_handle, message_queue):
    message_queue.put({"type":"on_complete"})

def download(url, message_queue):
    print(
        "Download process starting ... PID:{}, PPID:{}"
        .format(
            os.getpid(),
            psutil.Process(os.getpid()).ppid()), flush=True)
    on_progress_with_MQ = partial(on_progress, message_queue=message_queue)
    on_complete_with_MQ = partial(on_complete, message_queue=message_queue)
    youtube_clip = YouTube(
                        url,
                        on_progress_callback=on_progress_with_MQ,
                        on_complete_callback=on_complete_with_MQ)
    youtube_stream = youtube_clip.streams.get_highest_resolution()
    youtube_stream.download("videos")

- Multiprocessing

In [30]:
# 미국은 어떻게 강대국이 되었나
url = "https://www.youtube.com/watch?v=UbGppTooNho"

print("main process running ... PID:{}".format(os.getpid()), flush=True)

message_queue = multiprocessing.Queue()

p1 = multiprocessing.Process(target=draw_ui, args=(message_queue,))
p2 = multiprocessing.Process(target=download, args=(url, message_queue,))

p1.start()
p2.start()

p1.join()
p2.join()

main process running ... PID:20696
UI process starting ... PID:21391, PPID:20696
Download process starting ... PID:21394, PPID:20696


Downloading...: 100%|██████████| 100/100 [00:02<00:00, 47.89it/s]


Completed!
